In [1]:
from odt_parse import OdtData
from odt_diff import find_style_by_name

In [2]:
import csv, glob, zipfile

In [3]:
HOME_FOLDER = '/Users/ecervera/Google Drive/Docencia/Curs1718/PS1002/Examen Enero 1718/'
SUBMISSION_FOLDER = HOME_FOLDER + 'PS1002-2-20172018-Entrega ejercicios prácticos-3118842'
user_folder = glob.glob(SUBMISSION_FOLDER + '/*')

In [4]:
ref_name_v1 = HOME_FOLDER + 'solucionA_v1.odt'
ref_name_v2 = HOME_FOLDER + 'solucionA_v2.odt'

par_prop = ['backgroundcolor', 'textalign', 'marginleft', 'marginright', 'margintop', 'marginbottom', 'textindent']
text_prop = ['fontsize', 'fontstyle', 'fontfamily', 'fontrelief', 'fontvariant', 'fontweight']
page_prop=['nextstylename']

ref_v1 = OdtData(ref_name_v1, par_prop, text_prop, page_prop)
ref_v2 = OdtData(ref_name_v2, par_prop, text_prop, page_prop)

In [5]:
sp_dict = {'Heading':'Título', 'Heading_1':'Título_1', 'breakbefore':'salto de página antes',
           'backgroundcolor':'color de fondo', 'textalign':'alineación', 
           'Quotations':'Cita', 'fontsize':'tamaño de letra', 'fontstyle':'efecto tipográfico', 
           'marginleft':'sangría izquierda', 'marginright':'sangría derecha', 
           'margintop':'espacio superior', 'marginbottom':'espacio inferior',
           'justify':'justificada', 'end':'derecha', 'start':'izquierda',
           'nextstylename': 'estilo de página siguiente'}
def sp_trans(s):
    try:
        t = sp_dict[s]
    except KeyError:
        t = s
    return t

In [6]:
errors = []
def compare_style_attr(ref, doc, family, style_name, attr_list):
    stref = find_style_by_name(ref.style[family], style_name)
    stdoc = find_style_by_name(doc.style[family], style_name)
    if stdoc:
        for attr in attr_list:
            try:
                val_ref = stref[attr]
                try:
                    val_doc = stdoc[attr]
                    if val_ref != val_doc:
                        f.write('Estilo %s tiene %s \n  %s en lugar de %s.\n\n' % (sp_trans(style_name), sp_trans(attr),
                                                                                     sp_trans(val_doc), sp_trans(val_ref)))
                except KeyError:
                        f.write('Estilo %s no tiene %s definido.\n\n' % (sp_trans(style_name), sp_trans(attr)))
                #except TypeError:
                #        f.write('Estilo %s no está definido.\n\n' % (sp_trans(style_name)))
            except KeyError:
                err = style_name + "_" + attr
                if not err in errors:
                    errors.append(err)
                    print('Estilo %s no tiene %s definido en el fichero de referencia.' % (sp_trans(style_name), sp_trans(attr)))
    else:
        f.write('Estilo %s no está definido.\n\n' % (sp_trans(style_name)))

In [7]:
def compare_style_attrs_v1(ref, doc):
    compare_style_attr(ref, doc, 'page', 'miPágina', 
                       ['name'])
    compare_style_attr(ref, doc, 'page', 'miPortada', 
                       ['nextstylename'])
    compare_style_attr(ref, doc, 'paragraph', 'title', 
                       ['fontsize', 'marginbottom', 'margintop', 'textalign'])
    compare_style_attr(ref, doc, 'paragraph', 'subtitle', 
                       ['textalign'])
    compare_style_attr(ref, doc, 'paragraph', 'text_body', 
                       ['marginbottom', 'textalign', 'textindent'])
    compare_style_attr(ref, doc, 'paragraph', 'entradilla', 
                       ['fontstyle', 'margintop', 'marginleft','marginright', 
                        'marginbottom', 'textalign', 'parent'])
    compare_style_attr(ref, doc, 'paragraph', 'numbering_1', 
                       ['marginleft'])
    compare_style_attr(ref, doc, 'paragraph', 'list_1', 
                       ['marginleft', 'marginbottom'])
    compare_style_attr(ref, doc, 'paragraph', 'miprimerparrafo', 
                       ['textindent'])
    compare_style_attr(ref, doc, 'text', 'midestacado', 
                       ['fontfamily', 'fontweight'])

In [8]:
def compare_style_attrs_v2(ref, doc):
    
    compare_style_attr(ref, doc, 'page', 'páginaNormal', 
                       ['name'])
    compare_style_attr(ref, doc, 'page', 'primeraPágina', 
                       ['nextstylename'])
    compare_style_attr(ref, doc, 'paragraph', 'title', 
                       ['fontsize', 'marginbottom', 'margintop', 'textalign'])
    compare_style_attr(ref, doc, 'paragraph', 'subtitle', 
                       ['textalign'])
    compare_style_attr(ref, doc, 'paragraph', 'text_body', 
                       ['marginbottom', 'margintop', 'textalign', 'textindent'])
    compare_style_attr(ref, doc, 'paragraph', 'resumen', 
                       ['fontstyle', 'margintop', 'marginleft','marginright', 
                        'marginbottom', 'textalign', 'parent'])
    compare_style_attr(ref, doc, 'text', 'citadestacada', 
                       ['fontfamily', 'fontstyle'])
    compare_style_attr(ref, doc, 'text', 'autornombrado', 
                       ['fontfamily', 'fontweight'])

In [10]:
find_style_by_name(ref_v1.style['paragraph'], 'list_1')

{'marginbottom': '0.101cm',
 'marginleft': '1cm',
 'marginright': '0cm',
 'margintop': '0cm',
 'name': 'list_1',
 'parent': 'list',
 'textindent': '-0.64cm'}

In [10]:
find_style_by_name(ref_v2.style['text'], 'cita_destacada')

{'fontfamily': "'Liberation Sans Narrow'",
 'fontstyle': 'italic',
 'name': 'cita_destacada',
 'parent': None}

In [9]:
with open(SUBMISSION_FOLDER + '.txt', 'w', encoding='utf-8') as f:
    counter = 0
    for ufo in user_folder:
        user_files = glob.glob(ufo + '/*')
        for ufi in user_files:
            tokens = ufi.split('/')
            filename = tokens[-1]
            user_data = tokens[-2]
            user_name, user_id, _, submission_type, _ = user_data.split('_')
            fns = filename.split('.')
            if len(fns) > 1:
                extension = fns[-1]
            else:
                extension = ''
            row = [user_name, user_id, submission_type, filename, extension]
            #if filename != ref_name:
            #    f.write('\nNombre de fichero: %s\n\n' % filename)
            if extension == 'odt':
                f.write(user_name + '\n' + '-'*len(user_name) + '\n')
                doc = OdtData( ufi, par_prop, text_prop, page_prop )
                if doc.err:
                    f.write('Error de lectura de fichero\n')                   
                else:
                    if 'AV1' in filename:
                        f.write("Version 1\n\n")
                        compare_style_attrs_v1(ref_v1, doc)
                    elif 'AV2' in filename:
                        f.write("Version 2\n\n")
                        compare_style_attrs_v2(ref_v2, doc)
                    else:
                        f.write("Nombre incorrecto: version desconocida")
                        print("Error: nombre incorrecto, version desconocida")
                f.write('\n' + '#'*40 + '\n\n')
                counter += 1
            #else:
            #    f.write('Extensión de fichero incorrecta\n')
    print('%d ficheros procesados.' % counter)

101 ficheros procesados.
